# Preparación

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv("movimientos_bancarios.csv")
display(df.head())



meses = {
    "ene": "01", "feb": "02", "mar": "03", "abr": "04",
    "may": "05", "jun": "06", "jul": "07", "ago": "08",
    "sep": "09", "oct": "10", "nov": "11", "dic": "12"
}

df_fecha =df.copy()
# Cambio de abreviatura por número:
for esp, num in meses.items():
    df_fecha["Fecha_operacion"] = df_fecha["Fecha_operacion"].str.replace(esp,num, regex=False)

# Unirlo todo en formato reconocible para pandas (día, mes, año)
# Ej: "22 04 2025" lo transformamos a "22/04/2025"
df_fecha["Fecha_operacion"] = df_fecha["Fecha_operacion"].str.replace(" ", "/", n=2, regex=False)

# Convertimos ahora a datetime
df_fecha["Fecha_operacion"] = pd.to_datetime(df_fecha["Fecha_operacion"], format="%d/%m/%Y", errors="coerce")

#Comprobar que el cambio está bien realizado
df_fecha.info()
print(df_fecha["Fecha_operacion"].head())
df_fecha.sample()

df_fecha.loc[len(df_fecha)] = ['2025-03-31', 'Prestamo', 197.19, None]
df_fecha['Fecha_operacion'] = pd.to_datetime(df_fecha['Fecha_operacion'])  # asegurar formato datetime

df_fecha[df_fecha['Fecha_operacion'] == '2025-03-31']

# df_fecha.to_csv("Movimientos_fecha_corregida.csv", index=False)

,Fecha_operacion,Operacion,Importe,Saldo
0,22 abr 2025,Transferencia Inmediata De Evelymar Bastidas R...,15.0,840.16
1,22 abr 2025,Transferencia Inmediata De Evelymar Bastidas R...,75.0,825.16
2,22 abr 2025,Transferencia Inmediata De Evelymar Bastidas R...,300.0,750.16
3,21 abr 2025,"Pago Movil En Heladeria Quint, Jinamar, Tarj. ...",-10.6,450.16
4,21 abr 2025,Bizum A Favor De Ruth Santana Gonzalez Concept...,-10.0,460.76


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1535 entries, 0 to 1534
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fecha_operacion  1535 non-null   datetime64[ns]
 1   Operacion        1535 non-null   object        
 2   Importe          1535 non-null   float64       
 3   Saldo            1535 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 48.1+ KB
0   2025-04-22
1   2025-04-22
2   2025-04-22
3   2025-04-21
4   2025-04-21
Name: Fecha_operacion, dtype: datetime64[ns]


C:\Users\Gamer\AppData\Local\Temp\ipykernel_30748\57653331.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fecha.loc[len(df_fecha)] = ['2025-03-31', 'Prestamo', 197.19, None]


,Fecha_operacion,Operacion,Importe,Saldo
37,2025-03-31,"Compra Playstation Network, Hilversum, Tarjeta...",-13.99,1833.50
38,2025-03-31,Bizum De Debora Vega Ravelo Concepto Todo Saldado,160.00,1847.49
39,2025-03-31,"Pago Movil En The Barber Dog, La Mareta, Tarj....",-69.07,1687.49
40,2025-03-31,"Pago Movil En Panera, Valsequillo, Tarj. :*179057",-8.95,1756.56
41,2025-03-31,"Pago Movil En Lidl Can-gc-tel, Llanos De La, T...",-54.92,1765.51
42,2025-03-31,"Pago Movil En Hd Melenara, Telde, Tarj. :*179057",-59.30,1820.43
43,2025-03-31,"Pago Movil En Picoteo S. Migu, Telde, Tarj. :*...",-18.40,1879.73
1535,2025-03-31,Prestamo,197.19,NaN


In [3]:
# Añadir valor perdido 
df_fecha.loc[len(df_fecha)] = ['2025-03-31', 'Prestamo', 197.19, None]
df_fecha['Fecha_operacion'] = pd.to_datetime(df_fecha['Fecha_operacion'])  # asegurar formato datetime

df_fecha[df_fecha['Fecha_operacion'] == '2025-03-31']

C:\Users\Gamer\AppData\Local\Temp\ipykernel_30748\1795986240.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fecha.loc[len(df_fecha)] = ['2025-03-31', 'Prestamo', 197.19, None]


,Fecha_operacion,Operacion,Importe,Saldo
37,2025-03-31,"Compra Playstation Network, Hilversum, Tarjeta...",-13.99,1833.50
38,2025-03-31,Bizum De Debora Vega Ravelo Concepto Todo Saldado,160.00,1847.49
39,2025-03-31,"Pago Movil En The Barber Dog, La Mareta, Tarj....",-69.07,1687.49
40,2025-03-31,"Pago Movil En Panera, Valsequillo, Tarj. :*179057",-8.95,1756.56
41,2025-03-31,"Pago Movil En Lidl Can-gc-tel, Llanos De La, T...",-54.92,1765.51
42,2025-03-31,"Pago Movil En Hd Melenara, Telde, Tarj. :*179057",-59.30,1820.43
43,2025-03-31,"Pago Movil En Picoteo S. Migu, Telde, Tarj. :*...",-18.40,1879.73
1535,2025-03-31,Prestamo,197.19,NaN
1536,2025-03-31,Prestamo,197.19,NaN


In [4]:
# Unir los CSV
df_1 = pd.read_csv('movimientos_bancarios_extraidos.csv')
df_2 = pd.read_csv('Movimientos_fecha_corregida.csv')

df_unido = pd.concat([df_1, df_2], ignore_index=True)


# Convertimos ahora a datetime
df_unido['Fecha_operacion'] = pd.to_datetime(df_unido['Fecha_operacion'], format='%Y-%m-%d', errors='coerce')


display(df_unido.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2362 entries, 0 to 2361
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fecha_operacion  2362 non-null   datetime64[ns]
 1   Operacion        2362 non-null   object        
 2   Importe          2362 non-null   float64       
 3   Saldo            1847 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 73.9+ KB


None

In [5]:
# Corregimos nombres de las columnas
df_unido.columns = df_fecha.columns.str.lower().str.replace(" ", "_")
display(df_unido.head())

,fecha_operacion,operacion,importe,saldo
0,2021-09-23,TRANSFERENCIA DE EVELYMAR ESPERANZA BASTIDAS R...,175.0,NaN
1,2021-09-23,TRANSFERENCIA DE EVELYMAR ESPERANZA BASTIDAS R...,123.6,NaN
2,2021-09-23,TRANSFERENCIA DE EVELYMAR ESPERANZA BASTIDAS R...,55.0,NaN
3,2021-09-23,TRANSFERENCIA DE EVELYMAR ESPERANZA BASTIDAS R...,50.0,403.6
4,2021-09-24,"COMPRA PAGO M:VIL EN BURGER KING TOM, PALMAS (...",-9.0,NaN


In [6]:
# Arreglamos el contenido de los conceptos de las operaciones
def limpiar_texto(texto):
    texto = str(texto).lower()
    texto = re.sub(r'https?://\S+', '', texto)                  # quitar URLs
    texto = re.sub(r'[\d]+', '', texto)                         # quitar números
    texto = re.sub(r'[\*\.,:;/\-_"\'\(\)]+', ' ', texto)        # quitar símbolos comunes
    texto = re.sub(r'\s+', ' ', texto).strip()                  # quitar espacios repetidos
    return texto

df_unido['operacion_limpia'] = df_unido['operacion'].apply(limpiar_texto)
display(df_unido.sample(10))

,fecha_operacion,operacion,importe,saldo,operacion_limpia
1193,2024-10-04,"Pago Movil En Alcampo Telde, Telde, Tarj. :*17...",-29.36,1052.99,pago movil en alcampo telde telde tarj
2043,2023-10-30,"Pago Movil En Hd Melenara, Telde, Tarj. :*179057",-31.28,704.57,pago movil en hd melenara telde tarj
1977,2023-11-21,Transferencia De Gomez Bastidas Oscar Alejandr...,48.07,127.30,transferencia de gomez bastidas oscar alejandro
2264,2023-06-23,"Compra Internet En Repsol Waylet, Madrid, Tarj...",-36.07,21.12,compra internet en repsol waylet madrid tarj
82,2021-12-01,RECIBO LAS PALMAS VEGUETA N: RECIBO 0049 6140 ...,-29.90,NaN,recibo las palmas vegueta n recibo bbdsbcn ref...
555,2022-11-08,"PAGO MOVIL EN LA MARETA, TELDE, TARJ. :*585729",-8.48,291.26,pago movil en la mareta telde tarj
71,2021-11-25,"COMPRA PAGO M:VIL EN CAFETERIA AULAR, LAS PALM...",-1.30,NaN,compra pago m vil en cafeteria aular las palma...
79,2021-11-30,TRANSFERENCIA DE EVELYMAR ESPERANZA BASTIDAS R...,100.00,NaN,transferencia de evelymar esperanza bastidas r...
1005,2024-12-30,"Pago Movil En Leroy Merlin Te, Telde, Tarj. :*...",-24.43,1548.95,pago movil en leroy merlin te telde tarj
1238,2024-08-19,"Pago Movil En Hi Bubble Tea 1, Las Palmas De, ...",-4.50,844.60,pago movil en hi bubble tea las palmas de tarj


In [7]:
# Añadimos categoría ingreso / gasto
df_unido["tipo"] = df_unido["importe"].apply(lambda i: "ingreso" if i > 0 else "gasto")
display(df_unido.sample(3))

,fecha_operacion,operacion,importe,saldo,operacion_limpia,tipo
1736,2024-02-12,Bizum A Favor De Debora Vega Ravelo Concepto: ...,-20.0,723.45,bizum a favor de debora vega ravelo concepto p...,gasto
1203,2024-08-29,"Pago Movil En Pasteleria Yera, La Estrella, Ta...",-4.5,570.18,pago movil en pasteleria yera la estrella tarj,gasto
1756,2024-02-05,"Pago Movil En Agroquimica Jua, Telde, Tarj. :*...",-9.0,985.15,pago movil en agroquimica jua telde tarj,gasto


In [8]:
# Añadimos mes - año para poder agruparlos
df_unido["año_mes"] = df_unido["fecha_operacion"].dt.to_period("M").astype(str)
display(df_unido.sample(5))
df_unido.info()

,fecha_operacion,operacion,importe,saldo,operacion_limpia,tipo,año_mes
1912,2023-12-07,"Pago Movil En La Mareta, Telde, Tarj. :*179057",-7.96,734.25,pago movil en la mareta telde tarj,gasto,2023-12
1567,2024-04-11,"Pago Movil En Idea Market, Telde, Tarj. :*179057",-35.60,436.25,pago movil en idea market telde tarj,gasto,2024-04
360,2022-07-13,"COMPRA INTERNET EN ARTESIETE LAS T, HORNOS DEL...",-8.40,NaN,compra internet en artesiete las t hornos del ...,gasto,2022-07
618,2022-12-19,"PAGO MOVIL EN GLOBAL SALCAI, LAS PALMAS DE, TA...",-1.40,125.94,pago movil en global salcai las palmas de tarj,gasto,2022-12
149,2022-01-20,"COMPRA APPLE.COM/BILL, ITUNES.COM, TARJETA 516...",-4.99,156.96,compra apple com bill itunes com tarjeta comision,gasto,2022-01


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2362 entries, 0 to 2361
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   fecha_operacion   2362 non-null   datetime64[ns]
 1   operacion         2362 non-null   object        
 2   importe           2362 non-null   float64       
 3   saldo             1847 non-null   float64       
 4   operacion_limpia  2362 non-null   object        
 5   tipo              2362 non-null   object        
 6   año_mes           2362 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 129.3+ KB


In [9]:
# Eliminamos la columna saldo que no se encuentra en ambos datasets
df_unido = df_unido.drop(columns=["saldo"])

In [10]:
# lo guardamos en un nuevo CSV para trabajar sobre el nuevo formato datetime
df_unido.to_csv("Movimientos_limpios.csv", index=False)
df_verificacion = pd.read_csv("Movimientos_limpios.csv", parse_dates=['fecha_operacion'])
display(df_verificacion.sample(10))

,fecha_operacion,operacion,importe,operacion_limpia,tipo,año_mes
1595,2024-03-28,"Pago Movil En Cafeteria Paris, Telde, Tarj. :*...",-15.65,pago movil en cafeteria paris telde tarj,gasto,2024-03
1286,2024-07-29,"Pago Movil En Los Charcones P, Arucas, Tarj. :...",-3.00,pago movil en los charcones p arucas tarj,gasto,2024-07
2034,2023-10-30,"Compra Spotify P26cb3adc0, Stockholm, Tarjeta ...",-10.99,compra spotify pcbadc stockholm tarjeta comision,gasto,2023-10
1524,2024-04-29,"Transferencia De Evelymar Bastidas Rosario, Co...",60.40,transferencia de evelymar bastidas rosario con...,ingreso,2024-04
1372,2024-07-01,Traspaso: Ahorro Hucha Objetivo: Regalo,-15.10,traspaso ahorro hucha objetivo regalo,gasto,2024-07
2075,2023-10-10,Traspaso: Ahorro Hucha Objetivo: Regalo,-2.32,traspaso ahorro hucha objetivo regalo,gasto,2023-10
1907,2023-12-07,"Compra Playstation Network, Sony Psn, Tarjeta ...",-4.16,compra playstation network sony psn tarjeta co...,gasto,2023-12
2188,2023-07-28,Bizum De Debora Vega Ravelo Concepto .,40.00,bizum de debora vega ravelo concepto,ingreso,2023-07
1781,2024-01-26,"Compra Spotifyes P28f2dab82, Stockholm, Tarjet...",-14.99,compra spotifyes pfdab stockholm tarjeta comision,gasto,2024-01
1980,2023-11-20,"Compra Prime Video*hi6uu0l94, Primevideo.co, T...",-12.99,compra prime video hiuul primevideo co tarjeta...,gasto,2023-11


In [11]:
df_verificacion.isnull().sum()

fecha_operacion     0
operacion           0
importe             0
operacion_limpia    0
tipo                0
año_mes             0
dtype: int64

In [12]:
df_verificacion.columns

Index(['fecha_operacion', 'operacion', 'importe', 'operacion_limpia', 'tipo',
       'año_mes'],
      dtype='object')